# Importing the log model

In [ ]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.log.obj import EventLog,Event,Trace # under the hood objects within pm4py, subject to change
from pm4py import convert_to_event_log

# Load the CSV file into a pandas dataframe
df = pd.read_csv('Mobis_WS2425.csv', sep=';')

df["time:timestamp"] = pd.to_datetime(df["time:timestamp"])


print(df.head())

# Convert the dataframe to a pm4py log
event_log = convert_to_event_log(df, case_id_key="case:concept:name")

   case_id                                           activity  \
0      105                                file travel request   
1      105  check if travel request needs preliminary pric...   
2      105                    decide on approval requirements   
3      105                      check if booking is necessary   
4      105                   check if expense documents exist   

  case:travel_start case:travel_end                 timestamp  \
0        10.01.2017      15.01.2017 2017-01-17 11:17:00+00:00   
1        10.01.2017      15.01.2017 2017-01-17 11:23:00+00:00   
2        10.01.2017      15.01.2017 2017-01-17 11:24:00+00:00   
3        10.01.2017      15.01.2017 2017-01-17 11:24:00+00:00   
4        10.01.2017      15.01.2017 2017-01-18 05:59:00+00:00   

                type org:resource  cost  
0           Employee       JB8510   452  
1           Employee       JB8510     0  
2           Employee       JB8510     0  
3  Travel Department       KS9688     0  
4       

Exception: the specified case ID column is not contained in the dataframe. Available columns: ['activity', 'case:travel_end', 'case:travel_start', 'case_id', 'cost', 'org:resource', 'timestamp', 'type']